In [1]:
# # for logging 
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split


from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.loggers import TestTubeLogger
from pytorch_lightning.loggers import WandbLogger
import wandb

# For metrics
from pytorch_lightning import metrics

import math
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import io
import torchtext
from torchtext.utils import download_from_url, extract_archive
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import pytorch_lightning as pl
from pytorch_lightning.trainer.trainer import Trainer






from pytorch_lightning.callbacks.early_stopping import EarlyStopping # The EarlyStopping callback can be used to monitor a validation metric and stop the training when no improvement is observed.
"""
    To enable it:

    Import EarlyStopping callback.

    Log the metric you want to monitor using log() method.

    Init the callback, and set monitor to the logged metric of your choice.

    Pass the EarlyStopping callback to the Trainer callbacks flag.
"""

from pytorch_lightning import seed_everything
# seed_everything(42)

In [2]:
from pytorch_lightning.metrics import Metric
from pytorch_lightning.metrics.utils import _input_format_classification
from sklearn.metrics import classification_report
class MyClassificationReport(Metric):
    def __init__(self,threshold: float = 0.5,compute_on_step: bool = True,dist_sync_on_step: bool = False):
        super().__init__(
            compute_on_step=compute_on_step,
            dist_sync_on_step=dist_sync_on_step,
        )

        self.threshold = threshold
        self.add_state("preds", default=[], dist_reduce_fx=None)
        self.add_state("target", default=[], dist_reduce_fx=None)

        # rank_zero_warn(
        #     'Metric `MyClassificationReport` will save all targets and predictions in buffer.'
        #     ' For large datasets this may lead to large memory footprint.'
        # )

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        preds = preds.cpu()
        target = target.cpu()
        y_hat, y = preds.max(1).indices, target
        assert y_hat.shape == y.shape
        self.preds.append(y_hat)
        self.target.append(y)

    def compute(self):
        preds = torch.cat(self.preds, dim=0)
        target = torch.cat(self.target, dim=0)
        return classification_report(y_true=target,y_pred=preds)

In [3]:

class AlarmDataset(Dataset):
    def __init__(self,data,seq_len,batch_size):
        self.length = len(data)//seq_len # how much data i have         
        self.data = data
        self.seq_len = seq_len
        self.batch_size = batch_size
       
    def __getitem__(self, index: int):
        x = self.data[index*self.seq_len:(index*self.seq_len)+self.seq_len]
        y = self.data[1+index*self.seq_len:1+(index*self.seq_len)+self.seq_len]
        return x,y
    
    def __len__(self) -> int:
        return self.length

class MyDataModule(pl.LightningDataModule):
    
    def __init__(self, dir_path:str, file_name:str, config):
        super().__init__()
        # self.batch_size = batch_size
        # self.data_path = data_path
        self.config = config

        self.tokenizer = get_tokenizer('basic_english')
        self.vocab = build_vocab_from_iterator(map(self.tokenizer,iter(io.open(dir_path+file_name,encoding="utf8"))))
                
        # url = data_path
        # test_filepath, valid_filepath, train_filepath = extract_archive(download_from_url(url))
        seqs = None
        with open(dir_path+file_name) as f:
            seqs = f.readlines()
        seqs = [seq for seq in seqs if len(seq.split())<=self.config['filter-seq-len']]

        print(f"total seqs= {len(seqs)}")
        print(seqs[:4])
        train, valid = train_test_split(seqs,test_size=0.30,shuffle=False)
        valid, test = train_test_split(valid,test_size=0.30, shuffle=False)

        with open(dir_path +"train.tokens","w") as f:
            for seq in train:
                f.write(seq)
        
        with open(dir_path +"val.tokens","w") as f:
            for seq in valid:
                f.write(seq)
            
        with open(dir_path +"test.tokens","w") as f:
            for seq in test:
                f.write(seq)

        train_data = self.data_process(iter(io.open(dir_path +"train.tokens", encoding="utf8")))
        val_data = self.data_process(iter(io.open(dir_path +"val.tokens", encoding="utf8")))
        test_data = self.data_process(iter(io.open(dir_path +"test.tokens", encoding="utf8")))

    
        self.train_dataset = AlarmDataset(train_data, self.config['seq-len'], self.config['batch-size'])
        self.valid_dataset = AlarmDataset(val_data,self.config['seq-len'], self.config['batch-size'])
        self.test_dataset = AlarmDataset(test_data, self.config['seq-len'], self.config['batch-size'])

    
    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)],dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


    def prepare_data(self):
        """
            Use this method to do things that might write to disk or that need to be done only from a single GPU in distributed settings.
            e.g., download,tokenize,etc…
        """ 
        return None


    def setup(self, stage: None):
        """
            There are also data operations you might want to perform on every GPU. Use setup to do things like:
            count number of classes,build vocabulary,perform train/val/test splits,apply transforms (defined explicitly in your datamodule or assigned in init),etc…
        """
        return None

    def train_dataloader(self) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(self.valid_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.config['batch-size'], shuffle=False,num_workers=8,drop_last=True, pin_memory=True)



In [4]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [5]:
class TransformerModel(pl.LightningModule):

    def __init__(self, config):
        super(TransformerModel, self).__init__()
        self.config = config        
        self.lr = self.config["lr"]
        self.weight_decay = self.config["weight-decay"]
    
        self.pos_encoder = PositionalEncoding(self.config['em-size'], self.config['dropout'])
        encoder_layers = torch.nn.TransformerEncoderLayer(self.config['em-size'], self.config['nhead'], self.config['nhid'], self.config["dropout"])
        self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layers, self.config['nlayers'])
        self.encoder = torch.nn.Embedding(self.config["vocab-size"], self.config['em-size'])
        self.decoder = torch.nn.Linear(self.config['em-size'], self.config["vocab-size"])
        self.src_mask = self.generate_square_subsequent_mask(self.config['seq-len'])
        self.init_weights()

        class_weights = torch.ones(self.config['vocab-size']) * (1.0/self.config['vocab-size'])
        self.class_weights = class_weights.cuda()

        self.train_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])
        self.val_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])
        self.test_F1 = metrics.classification.F1(num_classes=self.config["vocab-size"])

        self.val_MCR = MyClassificationReport()
        self.test_MCR = MyClassificationReport()
        


        self.log("Sequence length",self.config['seq-len'])
        self.log("lr",self.lr)
        self.log("# of tokens/vocab_size (unique alarms)",self.config['vocab-size'])
        self.log("weight_decay",self.weight_decay)
        self.save_hyperparameters()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src_mask = src_mask.to(self.device)
        src = self.encoder(src) * math.sqrt(self.config['em-size'])
        src = self.pos_encoder(src)
      
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        
        return output

   # The ReduceLROnPlateau scheduler requires a monitor
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr,weight_decay=self.weight_decay)
        d = {
       'optimizer': optimizer,
       'lr_scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", factor = 0.5, patience=8, verbose=True),
       'monitor': 'val_epoch_loss',
        'interval': 'epoch'
        }
        return d    
    
    def training_step(self,batch,batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)

        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1, self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y, weight=self.class_weights) # cross entropy itself compute softmax 
        
        self.log('train_loss',loss,logger=True)
        self.log('train_F1',self.train_F1(F.softmax(y_hat),y),logger=True, on_step=True,prog_bar=True)
        return loss

    def validation_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)
        
        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))
        
        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1, self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y,weight=self.class_weights)

        self.val_MCR(F.softmax(y_hat) ,y)

        self.log('val_loss',loss,logger=True)
        self.log('val_F1',self.val_F1(F.softmax(y_hat) ,y),logger=True, prog_bar=True)
        return {'val_loss':loss}
    
    def test_step(self,batch, batch_idx):
        x,y = batch
        x = x.T
        y = y.T.reshape(-1)
        if x.size(0) != self.config['seq-len']:
           self.src_mask =  self.generate_square_subsequent_mask(x.size(0))

        y_hat = self(x,self.src_mask)
        y_hat =  y_hat.view(-1,  self.config['vocab-size'])
        loss = F.cross_entropy(y_hat,y,weight=self.class_weights)

        self.test_F1(F.softmax(y_hat) ,y)
        self.test_MCR(F.softmax(y_hat) ,y)
        self.log('test_loss',loss,logger=True)
        self.log('test_F1', self.test_F1(F.softmax(y_hat) ,y),logger=True)
        return {'test_loss':loss}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([d['loss']  for d in outputs]).mean()

        f1 = self.train_F1.compute()
        print(f">Epoch ={self.current_epoch}, Avg Training loss = {avg_loss}, F1 = {f1}")
        self.log("train_epoch_loss",avg_loss,logger=True,prog_bar=True)
        self.log("train_epoch_F1", f1, logger=True,prog_bar=True)
  
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([d['val_loss'] for d in outputs]).mean()
        # y_hats = torch.stack([d['y_hat'].cpu() for d in outputs])
        # ys = torch.stack([d['y'].cpu() for d in outputs])
        # ys = ys.view(-1)
        # y_hats = y_hats.view(-1,self.config['vocab-size'])

        # # print(f"Shape ys={ys.size()}, y_hats={y_hats.size()}")
        print("    **************** Validation Classification Report *****************")
        print(self.val_MCR.compute(),file=open("val-output.txt",'w'))

        f1_score = self.val_F1.compute()
        print(f">== Average Valid Loss = {avg_loss}, F1 = {f1_score}")
        self.log("val_epoch_loss",avg_loss,logger=True)
        self.log("val_epoch_F1",f1_score,logger=True,prog_bar=True)
        
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([d['test_loss'] for d in outputs]).mean()
        f1 = self.test_F1.compute()
        


        print(f">Average Test Loss = {avg_loss}, f1= {f1}")
        self.log("test_epoch_loss",avg_loss, logger = True)
        self.log("test_epoch_F1",f1, logger=True)
        # self.log("test_epoch_cm",cm, logger=True)

        print("    **************** Test Classification Report *****************")
        print(self.test_MCR.compute(),file=open("test-output.txt",'w'))
    

# Trainning

**Note: When monitoring any parameter after the validation epoch end then you should pass check_val_every_n_epoch=1  not to other. This is very important.**

In [6]:
dir_name = "../.data/"
fname = 'seqs.tokens'

# setup data
config_data = {
'batch-size' :16, # Batch Size 
'seq-len' : 700, # Sequence length
'filter-seq-len' :350 # remove sequence whose size is greater than this len
}

dm = MyDataModule(dir_path=dir_name,file_name=fname,config=config_data)


config_model = {
    'lr' : 0.0001,
    'dropout' : 0.1,
    'weight-decay': 0.0003,
    'em-size' :256, # embedding dimension 
    'nhid' : 266, # the dimension of the feedforward network model in nn.TransformerEncoder
    'nlayers' :3, # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    'nhead' : 8, # the number of heads in the multiheadattention models
    'seq-len': config_data['seq-len'], # dont use wandb config 
    'vocab-size':len(dm.vocab.stoi) # the size of vocabulary /also called tokens
}

# setup model - note how we refer to sweep parameters with wandb.config
model = TransformerModel(config=config_model)


30825lines [00:02, 14111.43lines/s]
total seqs= 27975
['A17 A75 A17 A57 A17 A99 A98 A56\n', 'A245 A246 A50 A243\n', 'A243 A9 A559 A1025\n', 'A50 A59 A60 A64 A392 A726 A9 A725 A726 A725 A243 A725\n']


## Loggers

In [7]:
wandb_logger = WandbLogger(project="Alarm-Transformers-Net")
comet_logger = CometLogger(
    api_key="YZWScOiWdE8FwQSUj725dRmor",
    project_name="Alarm-Transformers-Net" # Optional
)
test_tube_logger = TestTubeLogger('tb_logs', name='Alarm-Transformers-Net')

CometLogger will be initialized in online mode


In [8]:
early_stop_callback = EarlyStopping(
   monitor='val_epoch_loss',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='min'
)

# setup Trainer
trainer = Trainer(precision=16,gpus=-1, num_nodes=1,  max_epochs=1, check_val_every_n_epoch=10,deterministic=True, logger=[wandb_logger] ,gradient_clip_val=0.5,enable_pl_optimizer=True,callbacks=[early_stop_callback],progress_bar_refresh_rate =0)
trainer.fit(model,dm) # traning and validation


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: waris (use `wandb login --relogin` to force relogin)



  | Name                | Type                   | Params
---------------------------------------------------------------
0 | pos_encoder         | PositionalEncoding     | 0     
1 | transformer_encoder | TransformerEncoder     | 1.2 M 
2 | encoder             | Embedding              | 245 K 
3 | decoder             | Linear                 | 246 K 
4 | train_F1            | F1                     | 0     
5 | val_F1              | F1                     | 0     
6 | test_F1             | F1                     | 0     
7 | val_MCR             | MyClassificationReport | 0     
8 | test_MCR            | MyClassificationReport | 0     
---------------------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
>== Average Valid Loss = 7.241314888000488, F1 = 0.00022321428696159273
    **************** Validation Classification Report *****************
>Epoch =0, Avg Training loss = 1.336412787437439, F1 = 0.792586326

MisconfigurationException: ReduceLROnPlateau conditioned on metric val_epoch_loss which is not available. Available metrics are: dict_keys(['train_F1', 'train_loss', 'train_epoch_loss', 'train_epoch_F1']). Condition can be set using `monitor` key in lr scheduler dict

In [ ]:
trainer.test(datamodule=dm) # testing

In [ ]:
# accelerator='dp'
# progress_bar_refresh_rate=0 # set to zero to disable it

In [ ]:
torch.ones(5) * 1/10